# ENGIE - GEMS 
## Evaluación de las habilidades de gestión de datos y digitalización. Conciliación datos

---
Prueba práctica del proceso de selección desarrollada por Alba Durán Merinero

## 1. Configuración de paths y librerías 

In [1]:
import funciones_caso_practico as f
import os

main_path = "datos/" 
facturas_path = main_path + "facturas_agiekey.xlsx"
mapping_path = main_path + "Mapping.xlsx"
deals_path = main_path + "Deals_Marzo_2024/"

## 2. Configuración de parámetros de la conciliación 

Es necesario establecer los criterios del análisis a llevar a cabo. Se necesita informar del Año, Mes, Origen y Servicio Facturado de las facturas a conciliar.
Toda la información debe aportarse en formato string (texto entrecomillado)

**IMPORTANTE:**
En este notebook se han configurado los parámetros para la resolución del apartado 2b.

In [2]:
#PARAMETROS PARA REPRODUCIR LOS RESULTADOS DEL APARTADO 2a
#Fecha
ano ='2024'
mes = '02'
#Origen y Servicio Facturado a conciliar, expresado igual que en el fichero Facturas_agiekey
origen = 'TVB'
servicio_facturado = 'Almacenamiento TVB'

## 3. Fichero facturas 

In [3]:
#Carga del fichero de facturas
facturas = f.carga_xlsx(facturas_path)
print('Número total de facturas: ' + str(facturas.shape[0]))

Número total de facturas: 14


In [4]:
#Visualización df facturas inicial
facturas.head(2)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,Estado,FechaEstado,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones
0,2024264672,ESA86484292,"ENAGAS GTS, SAU",60,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,123195.13,EUR,Recibida,4/3/24,4/3/24,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN
1,2024264671,ESA86484292,"ENAGAS GTS, SAU",60,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,111963.62,EUR,Recibida,4/3/24,4/3/24,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN


### 3.1 Filtrado de facturas

In [5]:
#Filtro de Facturas según el Origen y Servicio Facturado a conciliar
facturas = facturas[(facturas['Origen'] == origen) & (facturas['ServicioFacturado'] == servicio_facturado)]
print('Número de facturas correspondientes al Origen y Servicio Facturado: ' + str(facturas.shape[0]))

Número de facturas correspondientes al Origen y Servicio Facturado: 11


In [6]:
#Visualización df facturas filtrando el concepto de Origen y Servicio Facturado correspondiente
facturas.head(2)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,Estado,FechaEstado,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones
3,2024164592,ESA86484292,"ENAGAS GTS, SAU",50,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,1375.12,EUR,Recibida,4/3/24,4/3/24,eMail,344991,TVB,Almacenamiento TVB,NaN,NaN
4,2024164591,ESA86484292,"ENAGAS GTS, SAU",50,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,1787.43,EUR,Recibida,4/3/24,4/3/24,eMail,344991,TVB,Almacenamiento TVB,NaN,NaN


### 3.2 Resumen de facturas

In [7]:
#Resumen de facturas
resumen_fact = f.global_info(facturas)

#Prints de la información
print('Número de facturas correspondientes al Origen y Servicio Facturado: ' + str(resumen_fact['NumeroFacturasValidas'][0]))
print('Número de clientes: ' + str(resumen_fact['NumeroClientes'][0]))
print('Facturación global del concepto: ' + str(resumen_fact['FacturacionTotal'][0]))

Número de facturas correspondientes al Origen y Servicio Facturado: 11
Número de clientes: 1
Facturación global del concepto: 72435.76EUR


## 4. Carga del fichero de mapping 

In [8]:
#Carga fichero mapping
mapping = f.carga_xlsx(mapping_path)
#visualización df mapping inicial 
mapping.head(2)

,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
1,TVB,PLANTAS,JVLNG,ICC,GETRA


In [9]:
#Filtro de dataset Mapping según el Origen y Servicio Facturado a conciliar en Facturas
mapping = mapping[(mapping['Origen'] == origen) & (mapping['Servicio facturado'] == servicio_facturado)]
#Creación nueva columna
mapping = f.combcols(mapping, 'FileName', ['Portfolio','Commodity','DealType'])
#En caso de filas 100% duplicadas se mantiene solo la primera
mapping = mapping.drop_duplicates(keep='first')
#Visualización del df mapping filtrando el concepto de Origen y Servicio Facturado correspondiente
mapping.head(2)

,Origen,Servicio facturado,Portfolio,Commodity,DealType,FileName
16,TVB,Almacenamiento TVB,JVLNG,CTV,STOK,JVLNG_CTV_STOK


### 4.1 Selección a través del mapping del archivo de Deals corresponiente para conciliar las facturas

In [10]:
#Lista de archivos que coinciden según el origen
file_names = list(mapping['FileName'])
#Obtencion de lista de archivos en el directorio de los deals
list_files = os.listdir(deals_path)
#Filtro de archivos que correspondientes
filtered_files = [file for file in list_files if (file.startswith(file_names[0])) & (file.endswith('.xlsx'))]
#Construcción del path
fil_file_path = deals_path + filtered_files[0]


## 5. Carga fichero de deals

In [11]:
deals = f.carga_xlsx(fil_file_path)
print('Numero total de deals: '+ str(deals.shape[0]))
#Visualización df deals inicial
deals.head(2)

Numero total de deals: 42


,Unnamed: 0,AggregatedKey,ApplicationPeriod,BookingSource,BOValidated,BOValidationDate,BOValidator,CancelsDealId,Capacity,CapacityFrequency,...,ExerciseDate,ExerciseType,Nature,OptionNature,OptionType,PeriodInfoCollection,Script,TemplateId,TemplateScript,OrderId
0,0,"42563586,2024/05/20 13:35:59,GEFWD",010524,MET,True,2024-05-20T13:35:59+02:00,GGH410,0,90000,1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"42518127,2024/05/20 13:34:13,GEFWD",040624,MET,True,2024-05-20T13:34:13+02:00,GGH410,0,50000,1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 5.1 Selección de Deals del año de estudio

In [12]:
#Filtramos los deals correspondientes al año actual
deals['StartDeliveryDate'] = deals['StartDeliveryDate'].astype(str)
deals['EndDeliveryDate'] = deals['EndDeliveryDate'].astype(str)
deals = deals[(deals['StartDeliveryDate'].apply(lambda x: x.startswith(ano)) & (deals['EndDeliveryDate'].apply(lambda x: x.startswith(ano))))]
print('Numero de Deals del año de estudio: ' + str(deals.shape[0]))
#Visualización de df deals filtrando los datos del año de estudio
deals.head(2)

Numero de Deals del año de estudio: 26


,Unnamed: 0,AggregatedKey,ApplicationPeriod,BookingSource,BOValidated,BOValidationDate,BOValidator,CancelsDealId,Capacity,CapacityFrequency,...,ExerciseDate,ExerciseType,Nature,OptionNature,OptionType,PeriodInfoCollection,Script,TemplateId,TemplateScript,OrderId
0,0,"42563586,2024/05/20 13:35:59,GEFWD",010524,MET,True,2024-05-20T13:35:59+02:00,GGH410,0,90000,1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"42518127,2024/05/20 13:34:13,GEFWD",040624,MET,True,2024-05-20T13:34:13+02:00,GGH410,0,50000,1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 5.2 Análisis de los sub-flows
La información de cada uno de los flows que se engloban en un Deal, se puede obtener del campo ['Flow'], el cual se encuentra en formato json

In [13]:
#Extracción de información de sub-flows de los registros correspondientes
flow_df = f.extract_df_from_json_src(df= deals, col= 'Flows',deal_col= 'Id')
print('Número total de sub-flows: ' + str(flow_df.shape[0]))
#Visualización df flows inicial
flow_df.head(2)

Generando dataframe para deal:  42563586
Generando dataframe para deal:  42518127
Generando dataframe para deal:  42518126
Generando dataframe para deal:  42518125
Generando dataframe para deal:  42518124
Generando dataframe para deal:  42518092
Generando dataframe para deal:  42518091
Generando dataframe para deal:  42518090
Generando dataframe para deal:  42518089
Generando dataframe para deal:  42517580
Generando dataframe para deal:  42499606
Generando dataframe para deal:  42491259
Generando dataframe para deal:  42464106
Generando dataframe para deal:  42449534
Generando dataframe para deal:  42439259
Generando dataframe para deal:  42439258
Generando dataframe para deal:  42294515
Generando dataframe para deal:  42294514
Generando dataframe para deal:  42294493
Generando dataframe para deal:  42238975
Generando dataframe para deal:  42238679
Generando dataframe para deal:  42238678
Generando dataframe para deal:  42238677
Generando dataframe para deal:  42238676
Generando datafr

,Deal_Id,SubFlow_Id,Date,Subcantidad
0,42563586,108338130,2024-06-20T00:00:00+02:00,0.0
1,42518127,108217858,2024-06-10T00:00:00+02:00,0.0


In [14]:
#Selección de los flows del año y mes correspondientes a las facturas
flow_df = flow_df[flow_df['Date'].apply(lambda x: x.startswith(ano + '-' + mes))]
print('Número de sub-flows para análisis: ' + str(flow_df.shape[0]))
#Visualización df flows filtrando las fechas correspondientes al análisis
flow_df.head()

Número de sub-flows para análisis: 2


,Deal_Id,SubFlow_Id,Date,Subcantidad
27,39942100,100950845,2024-02-20T00:00:00+01:00,0.0
28,39942100,100950846,2024-02-20T00:00:00+01:00,0.0


## 6. Calculo de totales 
Calculos y operaciones necesarios para la obtención de información requerida por BackOffice

### 6.1 Facturas de AGIKEY

In [15]:
facturas[['NumeroFactura','Importe', 'FechaFactura']]

,NumeroFactura,Importe,FechaFactura
3,2024164592,1375.12,4/3/24
4,2024164591,1787.43,4/3/24
5,2024164590,1930.23,4/3/24
6,2024164589,1472.13,4/3/24
7,2024164588,1965.66,4/3/24
8,2024164587,1041.67,4/3/24
9,2024164586,1030.97,4/3/24
10,2024164585,1017.59,4/3/24
11,2024164584,1015.95,4/3/24
12,2024164583,718.6,4/3/24


In [16]:
#Cantidades total de las facturas
TotalQuantityAGIKEY = facturas['Importe'].astype('float').sum()

### 6.2 Deals TRS

In [17]:
TotalQuantityTRS = flow_df['Subcantidad'].sum()

In [18]:
results_deals = deals[['Id', 'StartDeliveryDate', 'EndDeliveryDate']]
results_deals['Cantidad'] = TotalQuantityTRS
results_deals['Procedencia TRS'] = file_names[0]
results_deals = results_deals[['Id', 'Cantidad', 'StartDeliveryDate', 'EndDeliveryDate', 'Procedencia TRS']]
results_deals = results_deals.rename(columns={'Id':'Deal'})
results_deals

,Deal,Cantidad,StartDeliveryDate,EndDeliveryDate,Procedencia TRS
0,42563586,0.0,2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00,JVLNG_CTV_STOK
1,42518127,0.0,2024-06-04T00:00:00+02:00,2024-06-04T00:00:00+02:00,JVLNG_CTV_STOK
2,42518126,0.0,2024-06-01T00:00:00+02:00,2024-06-01T00:00:00+02:00,JVLNG_CTV_STOK
3,42518125,0.0,2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00,JVLNG_CTV_STOK
4,42518124,0.0,2024-04-28T00:00:00+02:00,2024-04-28T00:00:00+02:00,JVLNG_CTV_STOK
5,42518092,0.0,2024-07-31T00:00:00+02:00,2024-07-31T00:00:00+02:00,JVLNG_CTV_STOK
6,42518091,0.0,2024-07-25T00:00:00+02:00,2024-07-25T00:00:00+02:00,JVLNG_CTV_STOK
7,42518090,0.0,2024-04-30T00:00:00+02:00,2024-04-30T00:00:00+02:00,JVLNG_CTV_STOK
8,42518089,0.0,2024-04-28T00:00:00+02:00,2024-04-28T00:00:00+02:00,JVLNG_CTV_STOK
9,42517580,0.0,2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00,JVLNG_CTV_STOK


In [19]:
flow_df

,Deal_Id,SubFlow_Id,Date,Subcantidad
27,39942100,100950845,2024-02-20T00:00:00+01:00,0.0
28,39942100,100950846,2024-02-20T00:00:00+01:00,0.0


### 6.2 Matcheo

In [20]:
Importe_conciliado = TotalQuantityTRS
Importe_no_conciliado = TotalQuantityAGIKEY - Importe_conciliado


## 7. Output final

In [21]:
print('('+ origen + ', ' + servicio_facturado +')')
print('---> FACTURAS DE AGIKEY <---')
facturas[['NumeroFactura','Importe', 'FechaFactura']].reset_index(drop=True)

(TVB, Almacenamiento TVB)
---> FACTURAS DE AGIKEY <---


,NumeroFactura,Importe,FechaFactura
0,2024164592,1375.12,4/3/24
1,2024164591,1787.43,4/3/24
2,2024164590,1930.23,4/3/24
3,2024164589,1472.13,4/3/24
4,2024164588,1965.66,4/3/24
5,2024164587,1041.67,4/3/24
6,2024164586,1030.97,4/3/24
7,2024164585,1017.59,4/3/24
8,2024164584,1015.95,4/3/24
9,2024164583,718.6,4/3/24


In [22]:
print('---> DEALS TRS <---')
results_deals

---> DEALS TRS <---


,Deal,Cantidad,StartDeliveryDate,EndDeliveryDate,Procedencia TRS
0,42563586,0.0,2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00,JVLNG_CTV_STOK
1,42518127,0.0,2024-06-04T00:00:00+02:00,2024-06-04T00:00:00+02:00,JVLNG_CTV_STOK
2,42518126,0.0,2024-06-01T00:00:00+02:00,2024-06-01T00:00:00+02:00,JVLNG_CTV_STOK
3,42518125,0.0,2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00,JVLNG_CTV_STOK
4,42518124,0.0,2024-04-28T00:00:00+02:00,2024-04-28T00:00:00+02:00,JVLNG_CTV_STOK
5,42518092,0.0,2024-07-31T00:00:00+02:00,2024-07-31T00:00:00+02:00,JVLNG_CTV_STOK
6,42518091,0.0,2024-07-25T00:00:00+02:00,2024-07-25T00:00:00+02:00,JVLNG_CTV_STOK
7,42518090,0.0,2024-04-30T00:00:00+02:00,2024-04-30T00:00:00+02:00,JVLNG_CTV_STOK
8,42518089,0.0,2024-04-28T00:00:00+02:00,2024-04-28T00:00:00+02:00,JVLNG_CTV_STOK
9,42517580,0.0,2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00,JVLNG_CTV_STOK


In [25]:
deal_list = list(results_deals['Deal'].unique())
for i in deal_list:
    aux_df = flow_df[flow_df['Deal_Id'] == i]
    aux_df = aux_df.reset_index(drop=True)
    if aux_df.empty:
        print('Deal '+ str(i) + ': sin flows asociados para el mes de estudio')
    else:
        print('Deal ' + str(i) + ': ')    
        print(aux_df[['SubFlow_Id','Date','Subcantidad']])

Deal 42563586: sin flows asociados para el mes de estudio
Deal 42518127: sin flows asociados para el mes de estudio
Deal 42518126: sin flows asociados para el mes de estudio
Deal 42518125: sin flows asociados para el mes de estudio
Deal 42518124: sin flows asociados para el mes de estudio
Deal 42518092: sin flows asociados para el mes de estudio
Deal 42518091: sin flows asociados para el mes de estudio
Deal 42518090: sin flows asociados para el mes de estudio
Deal 42518089: sin flows asociados para el mes de estudio
Deal 42517580: sin flows asociados para el mes de estudio
Deal 42499606: sin flows asociados para el mes de estudio
Deal 42491259: sin flows asociados para el mes de estudio
Deal 42464106: sin flows asociados para el mes de estudio
Deal 42449534: sin flows asociados para el mes de estudio
Deal 42439259: sin flows asociados para el mes de estudio
Deal 42439258: sin flows asociados para el mes de estudio
Deal 42294515: sin flows asociados para el mes de estudio
Deal 42294514:

In [ ]:
print(' -----------MATCHEO-------------')
print('TotalQuantityAGIKEY: ' + str(TotalQuantityAGIKEY) + ' --> Facturación Total')
print('TotalQuantityTRS: '+ str(TotalQuantityTRS) + ' --> Importe Conciliado')
print('TotalQuantityAGIKEY - TotalQuantityTRS: ' + str(TotalQuantityAGIKEY-TotalQuantityTRS) + ' --> Importe No Conciliado')

 -----------MATCHEO-------------
TotalQuantityAGIKEY: 72435.76000000001 --> Facturación Total
TotalQuantityTRS: 0.0 --> Importe Conciliado
TotalQuantityAGIKEY - TotalQuantityTRS: 72435.76000000001 --> Importe No Conciliado
